Assume a regression model $Y = X\beta + \epsilon$, where $\epsilon \sim \mathcal{N}(0,\sigma^2 I)$.

For the maximum likelihood estimator for the regression weights, $\beta$:

$P(y|\beta,\sigma^2) = \left( \frac{1}{\sigma\sqrt{2\pi}} \right)^N \exp\left\{ \frac{-1}{2\sigma^2} (Y-X\beta)^{T}(Y-X\beta) \right\}$

$\log P(y|\beta,\sigma^2) \propto (Y-X\beta)^{T}(Y-X\beta)$

$\frac{\partial \log P(y|\beta,\sigma^2)}{\partial \beta} = -2X^TY + 2X^TX\beta$

$\hat{\beta} = (X^TX)^{-1}X^TY$

---
And the noise parameter, $\sigma^2$.

$P(y|\beta,\sigma^2) = \left( \frac{1}{\sqrt{2\pi\sigma^2}} \right)^N \exp\left\{ \frac{-1}{2\sigma^2} (Y-X\beta)^{T}(Y-X\beta) \right\}$

$\log P(y|\beta,\sigma^2) \propto -\frac{N}{2}\log \sigma^2 - \frac{1}{2\sigma^2}(Y-X\beta)^{T}(Y-X\beta)$

$\frac{\partial \log P(y|\beta,\sigma^2)}{\partial \sigma^2} = -\frac{N}{2\sigma^2} + \frac{1}{2(\sigma^2)^2}(Y-X\beta)^{T}(Y-X\beta)$

$\hat{\sigma^2} = \frac{1}{N}\sum_{i=1}^{N}(Y_i - X_i\hat{\beta})^2$


In [9]:
import numpy as np

class LinearRegression(object):

    def __init__(self, X, Y):
        """
        The __init__ method is called anytime you instantiate a class. This function should take in two arguemnts:
            X: A NxD dimensional real valued numpy array represeting the covariates.
            y: A N dimensional real valued numpy array representing the targets.
        "self" is used to represent an instance of a class object, variables prepended with "self." can be accessed
        by other methods within the class. You can read more about how "self" works in a Python class here:

        https://www.tutorialspoint.com/self-in-python-class

        """
        self.X = X
        self.N, self.D = self.X.shape
        self.Y = Y
        assert(self.Y.size == self.N) # make sure y is the appropriate size

        # instantiate regression parameter objects
        self.beta = np.zeros(self.D)
        self.sigma2 = 1.

    def fit(self):
        """
        This function will estimate the parameters of the linear regression model, beta and sigma2
        """
        self.beta = np.linalg.inv(self.X.T.dot(self.X)).dot(self.X.T.dot(self.Y))
        self.sigma2 = ((self.Y - self.X.dot(self.beta)).T.dot(self.Y - self.X.dot(self.beta)))/ self.N  # == np.mean((self.Y - (self.X @ self.beta))**2) \\(self.N - self.D- 1)

    def predict(self, X_test):
        """
        This function will take in some test set data "X_test" and return a predicted value "Y_test"
        """
        assert(X_test.shape[1] == self.D)
        Y_test = X_test.dot(self.beta)
        return Y_test


train the linear regression on the "Boston Housing" dataset, which is a collection of median housing price data in the U.S. city of Boston and associated covariate data: http://lib.stat.cmu.edu/datasets/boston.

An intercept term (a column of "ones" for the input data) should be included. The intercept term represents the mean value of the response when all covariates are equal to zero. Split the dataset into a training set consisting of the first 400 observations and a test set consisting of the last 106 observations. Train the model on the training data and predict the median housing prices on the test set. Compute the mean squared error between the predicted housing prices and the actual housing prices.

In [43]:
boston = np.loadtxt("BostonHousing.csv", skiprows=1, delimiter=",")

X, Y = boston[:,:-1], boston[:,-1]     #

X = np.hstack((X, np.ones((X.shape[0],1)))) # data_intercept = np.insert(boston,-1,1,1)

X_train, X_test = X[:400], X[400:]     # X_train,  X_tru_test = data_intercept[:400,:-1],data_intercept[400:,:-1]
Y_train, Y_test = Y[:400], Y[400:]     # Y_train,  Y_tru_test = data_intercept[:400,-1],data_intercept[400:,-1]

LR = LinearRegression(X_train,Y_train)
LR.fit()
Y_pred = LR.predict(X_test)

MSE = ((Y_test - Y_pred).T.dot(Y_test - Y_pred)) / Y_pred.shape[0] # == np.mean((predicted_Y-test_Y)**2)  \\(X_train.shape[0]- X_train.shape[1] - 1)

print("Test MSE: %.2f" % MSE)

Test MSE: 37.89


Rank the weighted covariates from most important to least according to size of the regression coefficient.

In [44]:
LR.beta

array([-1.91246374e-01,  4.42289967e-02,  5.52207977e-02,  1.71631351e+00,
       -1.49957220e+01,  4.88773025e+00,  2.60921031e-03, -1.29480799e+00,
        4.84787214e-01, -1.54006673e-02, -8.08795026e-01, -1.29230427e-03,
       -5.17953791e-01,  2.86725996e+01])

In [45]:
covariates = np.loadtxt("BostonHousing.csv", max_rows=1, delimiter=",", dtype=str)
covariates = np.hstack((covariates[:-1],"intercept"))
print(covariates[np.argsort(np.abs(LR.beta))[::-1]])

['intercept' 'nox' 'rm' 'chas' 'dis' 'ptratio' 'lstat' 'rad' 'crim'
 'indus' 'zn' 'tax' 'age' 'b']


In [8]:
LR.sigma2

22.305225584163445